In [1]:
# import sys
# !{sys.executable} -m pip install numpy

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from difflib import SequenceMatcher


In [3]:
social_media = pd.read_excel('Data_Collection/ChinaSocialMedia20200928.xlsx')
twitter = pd.read_excel('Data_Collection/ChinaTwitter20200928.xlsx')

In [4]:
print('Number of Columns in Social media file ' + str(len(social_media.columns)))
print('Number of rows in Social media file ' + str(len(social_media)))
print('Number of columns in Twitter file ' + str(len(twitter.columns)))
print('Number of rows in Twitter file ' + str(len(twitter)))


Number of Columns in Social media file 74
Number of rows in Social media file 1258
Number of columns in Twitter file 38
Number of rows in Twitter file 231


# Questions To Carl Regarding Data 

### General Questions:
- What is it that we supposed to collect is it twitter and facebook
- What is the purpose of the rest of this data do i need to do someting with it or just the data collection 
- I found some language description columns that might be helpful for language but last time you mentioned something about language annotation ( what do you mean by that )
- Some columns are empty has no values in it what is the point of it being in there
- When they say language 1 , 2 , 3 does that mean first language , second language and so on 


## Create sheet with FB and TW handles with language field 

In [6]:
twitter_file_handle_columns = [c for c in list(twitter.columns) if  'handle' in c]
print('There are '+str(len(twitter_file_handle_columns))+ ' different handle types in  twitter file sheet')
twitter_file_handle = social_media.filter(items=twitter_file_handle_columns).fillna(0)
twitter_file_handle.astype(bool).sum(axis=0)

There are 9 different handle types in  twitter file sheet


Embassy Twitter handle with NA            70
Chinese Ambassador Twitter handle         36
Chinese Press Officer Twitter handle       1
Deputy chief of mission Twitter handle     2
Consul General Twitter handle              3
CG/Mission Twitter handle                  8
dtype: int64

## Handle Sources in social media file

In [7]:
# Generate list of columns that has the word handle in it 
social_media_handle_columns = [c for c in list(social_media.columns) if  'handle' in c]

In [8]:
print('There are '+str(len(social_media_handle_columns))+ ' different handle types in Social Media sheet')

There are 13 different handle types in Social Media sheet


In [9]:
# Fill the retrieved data with 0 instead of NaN
social_media_handles = social_media.filter(items=social_media_handle_columns).fillna(0)

### Number of rows in each type of handles 

In [11]:
social_media_handles.astype(bool).sum(axis=0)

Embassy Twitter handle with NA            70
Embassy Twitter handle without NA         70
Chinese Ambassador Twitter handle         36
Chinese Press Officer Twitter handle       1
Deputy chief of mission Twitter handle     2
Consul General Twitter handle              3
CG/Mission Twitter handle                  8
Consulate General Facebook handle          4
Ministry official Twitter handle           7
Media outlet Twitter handle               59
Media outlet Facebook handle               4
Institute Twitter handle                  47
Institute Facebook handle                 85
dtype: int64

## Page and link sources in social media file

In [12]:
social_media_page_link_columns = [c for c in list(social_media.columns) if   'Page' in c or   'link' in c ]
print('There are '+str(len(social_media_page_link_columns))+ ' different page and link types in Social Media sheet')

There are 11 different page and link types in Social Media sheet


In [13]:
# Fill the retrieved data with 0 instead of NaN
social_media_page_link = social_media.filter(items=social_media_page_link_columns).fillna(0)

### Number of rows in each type of link  and page

In [16]:
social_media_page_link.astype(bool).sum(axis=0)

Embassy website link                    172
Embassy Twitter link                    229
Chinese Ambassador Twitter link         227
Chinese Press Officer Twitter link      228
Deputy chief of mission Twitter link    228
Chinese Embassy Facebook Page Name       59
Main Facebook Page Link                  61
Consulate General website link           19
Institute link                          158
Facebook link                            85
Misc links                                4
dtype: int64

# Twitter Handles

In [17]:
# get the columns
twitter_handles_columns = [c for c in list(social_media.columns) if 'Twitter' in c and 'handle' in c  ]
print('There are '+str(len(twitter_handles_columns))+ ' different handle types  for twitter in Social Media sheet')
# get the data
twitter_handles = social_media.filter(items=twitter_handles_columns).fillna(0)


There are 10 different handle types  for twitter in Social Media sheet


### Number of rows in each type of handle

In [19]:
twitter_handles.astype(bool).sum(axis=0)

Embassy Twitter handle with NA            70
Embassy Twitter handle without NA         70
Chinese Ambassador Twitter handle         36
Chinese Press Officer Twitter handle       1
Deputy chief of mission Twitter handle     2
Consul General Twitter handle              3
CG/Mission Twitter handle                  8
Ministry official Twitter handle           7
Media outlet Twitter handle               59
Institute Twitter handle                  47
dtype: int64

# Facebook Sources

In [21]:
facebook_sources_columns = [c for c in list(social_media.columns) if 'Facebook' in c and 'handle' in c or 'Facebook' in c and 'link' in c or 'Facebook' in c and 'Page' in c]
print('There are '+str(len(facebook_sources_columns))+ ' different source types  for facebook in Social Media sheet')

facebook_sources = social_media.filter(items=facebook_sources_columns).fillna(0)
facebook_sources.astype(bool).sum(axis=0)


There are 6 different source types  for facebook in Social Media sheet


Chinese Embassy Facebook Page Name    59
Main Facebook Page Link               61
Consulate General Facebook handle      4
Media outlet Facebook handle           4
Institute Facebook handle             85
Facebook link                         85
dtype: int64

# Facebook Handles

In [22]:
facebook_handles_columns = [c for c in list(social_media.columns) if 'Facebook' in c and 'handle' in c ]
print('There are '+str(len(facebook_handles_columns))+ ' different handle types  for facebook in Social Media sheet')

facebook_handles = social_media.filter(items=facebook_handles_columns).fillna(0)
facebook_handles.astype(bool).sum(axis=0)


There are 3 different handle types  for facebook in Social Media sheet


Consulate General Facebook handle     4
Media outlet Facebook handle          4
Institute Facebook handle            85
dtype: int64

# Facebook Links and Pages

In [23]:
facebook_pages_links_columns = [c for c in list(social_media.columns) if 'Facebook' in c and 'link' in c or 'Facebook' in c and 'Page' in c ]
print('There are '+str(len(facebook_pages_links_columns))+ ' different page and link types  for facebook in Social Media sheet')
facebook_pages_links = social_media.filter(items=facebook_pages_links_columns).fillna(0)
facebook_pages_links.astype(bool).sum(axis=0)


There are 3 different page and link types  for facebook in Social Media sheet


Chinese Embassy Facebook Page Name    59
Main Facebook Page Link               61
Facebook link                         85
dtype: int64

# Consolidated FB and Twitter handles with Language
- I ignored all the links and the page names 


### Consolidated Twitter 

In [24]:
# i need this one in here because i will filter based on NaN
twitter_handles = social_media.filter(items=twitter_handles_columns)

In [25]:
Consolidated_twitter_handles = []
for c in twitter_handles_columns:
#     array of 10 columns contain the set of handles
    Consolidated_twitter_handles.append(list(twitter_handles[twitter_handles[c].notnull()][c]))
#     array of flattened handles 
Consolidated_falt_twitter_handles = [h for hs in Consolidated_twitter_handles for h in hs ]

In [36]:
twitter_handles_columns

['Embassy Twitter handle with NA',
 'Embassy Twitter handle without NA',
 'Chinese Ambassador Twitter handle',
 'Chinese Press Officer Twitter handle',
 'Deputy chief of mission Twitter handle',
 'Consul General Twitter handle',
 'CG/Mission Twitter handle',
 'Ministry official Twitter handle',
 'Media outlet Twitter handle',
 'Institute Twitter handle']

In [37]:
twitter_language_columns

['Embassy Twitter language 1',
 'Embassy Twitter language 2',
 'Embassy Twitter language 3',
 'Chinese Ambassador Twitter language 1',
 'Chinese Ambassador Twitter language 2',
 'Chinese Ambassador Twitter language 3',
 'Chinese Press Officer Twitter language 1',
 'Chinese Deputy chief of mission Twitter language 1',
 'Chinese Deputy chief of mission Twitter language 2',
 'Consul General Twitter language 1',
 'CG/Mission Twitter language 1',
 'CG/Mission Twitter language 2',
 'Ministry official language 1',
 'Ministry official language 2',
 'Media outlet Twitter language 1',
 'Institute Twitter language 1']

### Consolidated twitter language

In [29]:
# This function is used to combine handls columns with language columns to use them to combine handles and languages
# firt iterate over  handle columns then language columns and match between them using squence matcher 
# second combine them into array of tuple if they match then use defaultdict to convert them into dictionary

def combine_handle_col_language_col(handle_cols,lang_cols):
    column_lang = []
    for c in handle_cols:
        for l in lang_cols:
            s = SequenceMatcher(None,c,l)
            if s.ratio() > 0.60:
                column_lang.append((c,l))
            else:
                continue
    column_lang_map = defaultdict(list)
    for col, lang in column_lang:
        column_lang_map[col].append(lang)
    return column_lang_map


# This function is used to create dictionary that maps the handles to the languages assigned to it for example Ahmed handle is using Araic Ahmed:Arabic
# first iterate over the language_hadle_column map that maps each handle column to the corresponding language column zipped with consolidated handle list
# for each handle iterate over all the language columns that correspond to it and fetch the language if it exists and add that to array of tuples 
# convert that array of tuples to dictionary using defaultdict 
# This function require passing the data to it to be able to retrive all the corresponding language columns 

def combine_handle_language(column_lang_map,Consolidated_handles,data):
    col_lang_merg = []
    checked = []
    langs = []
    for c, hl in zip(column_lang_map.items(),Consolidated_handles):
        for h in hl:
            if h != '  ':
                for l in c[1]:
                    if h not in checked:
                        col_lang_merg.append((h,list(data[data[c[0]]==h][l].values)))
                checked.append(h)
    col_lang_combined = defaultdict(list)
    for col, lang in col_lang_merg:
        col_lang_combined[col].append(lang)
    return col_lang_combined



In [34]:
# Array of language columns 
twitter_language_columns = [c for c in list(social_media.columns) if 'Twitter' in c and 'language' in c or 'Ministry' in c and 'language' in c ]


In [35]:
twitter_handle_lang_col_map = combine_handle_col_language_col(twitter_handles_columns,twitter_language_columns)

In [41]:
twitter_combined_handle_lang = combine_handle_language(twitter_handle_lang_col_map,Consolidated_twitter_handles,social_media)


In [42]:
twitter_combnined_handle_lang_data = pd.DataFrame(dict(twitter_combined_handle_lang).items(), columns=['Twitter_Handle','Language_1_2_3'])
twitter_combnined_handle_lang_data.to_csv('Consolidated_tw_handles.csv')


In [33]:
twitter_combnined_handle_lang_data

NameError: name 'twitter_combnined_handle_lang_data' is not defined

### Consolidated Facebook 

#### The correction of facebook handle

In [ ]:
# Institute Facebook handle column is not actual handle the handle for this column is Facebook link 
del facebook_handles_columns[-1]
facebook_handles_columns.append('Facebook link')

In [ ]:
facebook_handles = social_media.filter(items=facebook_handles_columns)

In [ ]:
Consolidated_facebook_handles = []
for c in facebook_handles_columns:
#     array of 10 columns contain the set of handles
    Consolidated_facebook_handles.append(list(facebook_handles[facebook_handles[c].notnull()][c]))
#     array of flattened handles 
Consolidated_falt_facebook_handles = [h for hs in Consolidated_facebook_handles for h in hs ]

In [ ]:
Consolidated_falt_facebook_handles

In [ ]:
['https://www.facebook.com/'+h+'/' for  h in Consolidated_falt_facebook_handles if 'http' not in h]

In [ ]:
Consolidated_falt_facebook_handles

### Consolidated facebook language

In [ ]:
# Array of language columns 
facebook_language_columns = [c for c in list(social_media.columns) if 'Facebook' in c and 'language' in c]


In [ ]:
# Combine facebook handle columns with facebook language columns 
facebook_handle_lang_col_map = combine_handle_col_language_col(facebook_handles_columns,facebook_language_columns)

In [ ]:
# Comnbine facebook handles with the corresponding language between 1 to 3

facebook_combined_handle_lang = combine_handle_language(facebook_handle_lang_col_map,Consolidated_facebook_handles,social_media)


In [ ]:
# Comnbine twitter handles with the corresponding language between 1 to 3

facebook_combined_handle_lang = combine_handle_language(facebook_handle_lang_col_map,Consolidated_facebook_handles,social_media)


In [ ]:
facebook_combnined_handle_lang_data = pd.DataFrame(dict(facebook_combined_handle_lang).items(), columns=['Facebook_Handle','Language_1_2_3'])
facebook_combnined_handle_lang_data.to_csv('Consolidated_fb_handles.csv')


In [ ]:
facebook_combnined_handle_lang_data['Facebook_Handle']

### Combined Twitter and Facebook 

In [ ]:
print('Number of handles in twitter without redunduncy '+ str(len(twitter_combnined_handle_lang_data)))
print('Number of handles in facebook without redunduncy '+ str(len(facebook_combnined_handle_lang_data)))
df1 = twitter_combnined_handle_lang_data
df2 = facebook_combnined_handle_lang_data

In [ ]:
combined_twitter_facebook_handles = pd.concat([df1, df2], axis=1).fillna(0)

In [ ]:
modDfObj = combined_twitter_facebook_handles.append(pd.Series(['ConfuciusGSU', 0, '0', '0'], index=combined_twitter_facebook_handles.columns ), ignore_index=True)


In [ ]:
combined_twitter_facebook_handles.to_csv('Combined_twitter_facebook_handles.csv')
modDfObj.to_csv('Combined_twitter_facebook_handles_extra_handle.csv')

In [ ]:
# Combine twitter handle columns with twitter language columns 
# column_lang = []
# for c in twitter_handles_columns:
#     for l in twitter_language_columns:
#         s = SequenceMatcher(None,c,l)
#         if s.ratio() > 0.60:
#             column_lang.append((c,l))
#         else:
#             continue
# column_lang_map = defaultdict(list)
# for col, lang in column_lang:
#     column_lang_map[col].append(lang)


In [ ]:
# Comnbine twitter handles with the corresponding language between 1 to 3
# col_lang_merg = []
# checked = []
# langs = []
# for c, hl in zip(column_lang_map.items(),Consolidated_twitter_handles):
#     for h in hl:
#         if h != '  ':
#             for l in c[1]:
#                 if h not in checked:
#                     if  np.nan not in list(social_media[social_media[c[0]]==h][l].values):
#                         col_lang_merg.append((h,list(social_media[social_media[c[0]]==h][l].values)))
#             checked.append(h)
# col_lang_combined = defaultdict(list)
# for col, lang in col_lang_merg:
#     col_lang_combined[col].append(lang)
